# Running the pre-night briefing dashboard within a notebook

## Notebook perparation

### Load jupyter extensions

In [1]:
#%load_ext lab_black
%load_ext autoreload
%autoreload 1

### Imports

Use `aimport` for `schedview` imports for ease of debugging.

In [2]:
import warnings
import math
import logging
from pathlib import Path
import panel as pn
import numpy as np
import pandas as pd
import param
import bokeh
from copy import deepcopy
import datetime
from pytz import timezone
import lzma
import pickle
import yaml
import json
from collections import OrderedDict
from tempfile import TemporaryDirectory, NamedTemporaryFile

In [3]:
import rubin_scheduler

In [4]:
rubin_scheduler.__version__

'3.4.0'

In [5]:
from astropy.time import Time, TimeDelta
from zoneinfo import ZoneInfo
import matplotlib as mpl
import matplotlib.pyplot as plt
import hvplot.pandas

In [6]:
from rubin_scheduler.scheduler.example import example_scheduler
from rubin_scheduler.scheduler import sim_runner
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_scheduler.scheduler.utils import SchemaConverter

In [7]:
%aimport schedview
%aimport schedview.app.prenight
%aimport schedview.compute.scheduler
%aimport schedview.collect.opsim
from schedview.plot.visitmap import BAND_COLORS

/Users/edennihy/repos/rubin_scheduler/rubin_scheduler/utils/constants.py:11: FutureWarning: Function survey_start_mjd is deprecated, use SURVEY_START_MJD.
  warnings.warn("Function survey_start_mjd is deprecated, use SURVEY_START_MJD.", FutureWarning)


In [8]:
import healpy as hp

### Further preparation of the notebook

In [9]:
pn.extension("terminal")

### Filter warnings

Several dependencies throw prodigious instances of (benign) warnings.
Suppress them to avoid poluting the executed notebook.

In [10]:
warnings.filterwarnings(
    "ignore",
    module="astropy.time",
    message="Numerical value without unit or explicit format passed to TimeDelta, assuming days",
)
warnings.filterwarnings(
    "ignore",
    module="pandas",
    message="In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="divide by zero encountered in divide",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="invalid value encountered in multiply",
)
warnings.filterwarnings(
    "ignore",
    module="holoviews",
    message="Discarding nonzero nanoseconds in conversion.",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="invalid value encountered in arcsin",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="All-NaN slice encountered",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim.scheduler.utils",
    message="invalid value encountered in cast",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim.scheduler.core_scheduler",
    message="All-NaN axis encountered",
)

## Configuration and initial configuration

Setting `keep_rewards` to `True` results in a dashboard that includes plots of rewards.

In [11]:
keep_rewards = True

Set the start date, scheduler, and observatory for the night:

In [12]:
from feb_2025_fbs_spec_survey import get_scheduler

In [13]:
nside, scheduler = get_scheduler()

In [14]:
from rubin_scheduler.scheduler.example import get_ideal_model_observatory

In [15]:
observatory = get_ideal_model_observatory(nside=nside, wind_speed=0, wind_direction=0)

Set `evening_mjd` to the integer calendar MJD of the local calendar day on which sunset falls on the night of interest.

In [16]:
evening_iso8601 = "2024-02-20"

night_date = datetime.date.fromisoformat(evening_iso8601)
evening_mjd = Time(evening_iso8601).mjd
night_date, evening_mjd

evening_end_iso8601 = "2024-02-20"

night_end_date = datetime.date.fromisoformat(evening_end_iso8601)
evening_end_mjd = Time(evening_end_iso8601).mjd
night_end_date, evening_end_mjd

(datetime.date(2024, 2, 20), 60360.0)

If we just use this day as the start and make the simulation duration 1 day, the begin and end of the simulation will probably begin in the middle on one night and end in the middle of the next.
Instead, find the sunset and sunrise of the night we want using the almanac, and use these to determine our start time and duration.

In [17]:
# If the date represents the local calendar date at sunset, we need to shift by the longitude in units of days
this_night = (
    np.floor(observatory.almanac.sunsets["sunset"] + observatory.site.longitude / 360)
    == evening_mjd
)

mjd_start = observatory.almanac.sunsets[this_night]["sun_n12_setting"][0]
mjd_end = observatory.almanac.sunsets[this_night]["sunrise"][0]

end_night = (
    np.floor(observatory.almanac.sunsets["sunset"] + observatory.site.longitude / 360)
    == evening_end_mjd
)

mjd_end_start = observatory.almanac.sunsets[end_night]["sun_n12_setting"][0]
mjd_end_end = observatory.almanac.sunsets[end_night]["sunrise"][0]

night_duration = mjd_end - mjd_start
time_start = Time(mjd_start, format="mjd")-TimeDelta(2./24.)
time_start.iso, night_duration

('2024-02-20 22:20:11.553', 0.4244780265726149)

In [18]:
night_duration

0.4244780265726149

In [19]:
observatory = ModelObservatory(mjd_start=mjd_start, nside=nside)

Minor adjustments to improve AuxTel survey estimations

In [20]:
observatory.setup_camera(filter_changetime=5.0, maxspeed=1.5)
observatory.setup_telescope(altitude_maxspeed=1.5, azimuth_maxspeed=1.5, settle_time=5.0)
observatory.setup_dome(azimuth_maxspeed=3.0)
# Assuming original `self.downtimes` has fields "start" and "end" of type float
dtype = [("start", float), ("end", float)]

# Replace with an empty structured array with the same fields
observatory.downtimes = np.array([], dtype=dtype)

Record the date of local day in the evening. 

## Run a simulation and create the app instance

For this example, simulate starting the default first day of observing:

In [21]:
mjd_start

60361.014022598974

In [22]:
conditions = observatory.conditions

In [23]:
night_duration

0.4244780265726149

In [24]:
if not keep_rewards:
    observatory, scheduler, observations = sim_runner(
        observatory, scheduler, survey_length=night_duration
    )
else:
    scheduler.keep_rewards = True
    observatory, scheduler, observations, reward_df, obs_rewards = sim_runner(
        observatory,
        scheduler,
        sim_duration=night_duration/2.,
        record_rewards=True,
    )

/Users/edennihy/repos/rubin_scheduler/rubin_scheduler/scheduler/model_observatory/kinem_model.py:762: UserWarning: 4 exposures in 30 seconds is violating number of shutter motion limit
  warnings.warn(msg)


progress = 39.57%

/Users/edennihy/repos/rubin_scheduler/rubin_scheduler/scheduler/model_observatory/kinem_model.py:762: UserWarning: 4 exposures in 30 seconds is violating number of shutter motion limit
  warnings.warn(msg)


progress = 83.29%

/Users/edennihy/repos/rubin_scheduler/rubin_scheduler/scheduler/model_observatory/kinem_model.py:762: UserWarning: 4 exposures in 30 seconds is violating number of shutter motion limit
  warnings.warn(msg)


Skipped 0 observations
Flushed 0 observations from queue for being stale
Completed 52 observations
ran in 0 min = 0.0 hours


## Save the simulation

In [25]:
data_dir = TemporaryDirectory()

In [26]:
import os

In [27]:
with NamedTemporaryFile(prefix="opsim-", suffix=".db", dir=data_dir.name) as temp_file:
    opsim_output_fname = temp_file.name

opsim_output_fname=('/Users/edennihy/repos/rubin_sim_outputs/opsim.db')
if os.path.exists(opsim_output_fname):
    os.remove(opsim_output_fname)

SchemaConverter().obs2opsim(observations, filename=opsim_output_fname)
opsim_output_fname

'/Users/edennihy/repos/rubin_sim_outputs/opsim.db'

In [28]:
with NamedTemporaryFile(
    prefix="scheduler-", suffix=".pickle.xz", dir=data_dir.name
) as temp_file:
    scheduler_fname = temp_file.name

scheduler_fname=('/Users/edennihy/repos/rubin_sim_outputs/scheduler.pickel.xz')
if os.path.exists(scheduler_fname):
    os.remove(scheduler_fname)

with lzma.open(scheduler_fname, "wb", format=lzma.FORMAT_XZ) as pio:
    pickle.dump(scheduler, pio)

scheduler_fname

'/Users/edennihy/repos/rubin_sim_outputs/scheduler.pickel.xz'

In [29]:
if keep_rewards:
    with NamedTemporaryFile(
        prefix="rewards-", suffix=".h5", dir=data_dir.name
    ) as temp_file:
        rewards_fname = temp_file.name
    
    rewards_fname=('/Users/edennihy/repos/rubin_sim_outputs/rewards.h5')
    if os.path.exists(rewards_fname):
        os.remove(rewards_fname)
    
    reward_df.to_hdf(rewards_fname, "reward_df")
    obs_rewards.to_hdf(rewards_fname, "obs_rewards")
    
    rewards_fname

/var/folders/mx/y8jt2rr901762gj1cml0v9sh0000gp/T/ipykernel_49616/2318355145.py:11: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['basis_function', 'basis_function_class', 'basis_weight', 'tier_label',
       'survey_label', 'survey_class'],
      dtype='object')]

  reward_df.to_hdf(rewards_fname, "reward_df")


If you're host doesn't have a lot of memory, you may need to clean out some memory before trying to start the dashboard.

## Make some custom plots

Make some custom plots, trying them out in the notebook before we define a file with which they can be added to the dashboard.

Get the observations in the same form the custom plotter will see them:

In [30]:
visits = schedview.collect.opsim.read_opsim(opsim_output_fname)

In [31]:
hvplot_kwargs_slew = {
    "kind": "scatter",
    "x": "slewDistance",
    "y": "slewTime",
    "ylabel": "slew time (seconds)",
    "xlabel": "slew distance (degrees)",
    "color": "visitTime",
    "size": "visitExposureTime",
    "clabel": "visit time (seconds)",
    "cmap": "isolum",
}
visits.hvplot(**hvplot_kwargs_slew)

:Scatter   [slewDistance]   (slewTime,visitTime,visitExposureTime)

In [32]:
hvplot_kwargs_airmass_hist = {
    "kind": "hist",
    "y": "airmass",
    "by": "note",
    "bins": 8,
    "height": 512,
    "ylabel": "number of visits",
}
visits.hvplot(**hvplot_kwargs_airmass_hist)

:NdOverlay   [note]
   :Histogram   [airmass]   (airmass_count)

Put these settings into json file that can be read by the dashboard:

In [33]:
hvplot_kwargs_airmass_plot = {
    "kind": "scatter",
    "x": "start_date",
    "y": "altitude",
    "ymin": 20,
    "ymax": 90,
    "by": "science_program",
}
visits.hvplot(**hvplot_kwargs_airmass_plot)

:NdOverlay   [science_program]
   :Scatter   [start_date]   (altitude)

In [34]:
with NamedTemporaryFile(
    prefix="custom_prenight_tabs-", suffix=".json", dir=data_dir.name
) as temp_file:
    custom_tabs_fname = temp_file.name

with open(custom_tabs_fname, "w") as custom_tabs_file:
    custom_json = json.dump(
        [
            {"name": "Slew Time", "settings": hvplot_kwargs_slew},
            {"name": "Airmass histogram", "settings": hvplot_kwargs_airmass_hist},
            {"name": "Altitude Plot by Target", "settings": hvplot_kwargs_airmass_plot},
        ],
        indent=4,
        fp=custom_tabs_file,
    )

Read it back to look at the contents:

In [35]:
visits

,fieldRA,fieldDec,observationStartMJD,visitExposureTime,filter,rotSkyPos,rotSkyPos_desired,numExposures,airmass,seeingFwhm500,...,moonAz,sunAz,moonRA,moonDec,moonDistance,solarElong,moonPhase,observation_reason,science_program,start_date
observationId,,,,,,,,,,,,,,,,,,,,,
0,85.775232,-23.280874,60361.014146,30.0,r,97.380247,0.0,4,1.007564,0.578955,...,31.059610,249.711584,116.118318,27.236920,58.401755,105.449351,78.486146,,BLOCK-305,2024-02-21 00:20:22.249013248+00:00
1,95.802500,-37.691111,60361.015661,420.0,r,216.454563,0.0,1,1.021096,0.609740,...,30.604249,249.402199,116.131546,27.235238,67.668892,107.386080,78.491822,,BLOCK-T359_WD_0621-376,2024-02-21 00:22:33.120884224+00:00
2,95.802500,-37.691111,60361.020607,420.0,r,223.322329,0.0,1,1.016605,0.546401,...,29.097682,248.384149,116.174430,27.229533,67.674789,107.384079,78.510245,,BLOCK-T359_WD_0621-376,2024-02-21 00:29:40.449364992+00:00
3,95.802500,-37.691111,60361.025554,420.0,r,230.328980,0.0,1,1.013616,0.540673,...,27.563663,247.347002,116.216915,27.223492,67.680284,107.382078,78.528535,,BLOCK-T359_WD_0621-376,2024-02-21 00:36:47.870502656+00:00
4,95.802500,-37.691111,60361.030512,420.0,r,238.698393,0.0,1,1.011318,0.620427,...,26.001749,246.280002,116.259162,27.217101,67.685404,107.380073,78.546767,,BLOCK-T359_WD_0621-376,2024-02-21 00:43:56.200104704+00:00
5,86.500000,-32.306444,60361.036111,300.0,r,339.472702,0.0,1,1.004595,0.608641,...,24.205790,245.040802,116.306482,27.209476,65.929550,102.838596,78.567242,,BLOCK-312,2024-02-21 00:51:59.974949888+00:00
6,122.554238,-36.159018,60361.041144,35.0,r,185.649522,0.0,1,1.099348,0.583235,...,22.560628,243.895355,116.348661,27.202253,63.630665,124.586749,78.585541,,BLOCK-306,2024-02-21 00:59:14.848632320+00:00
7,122.560925,-36.190501,60361.041624,35.0,r,186.264630,0.0,1,1.094505,0.583235,...,22.402267,243.784620,116.352663,27.201547,63.661496,124.565141,78.587280,,BLOCK-306,2024-02-21 00:59:56.290612224+00:00
8,122.555987,-36.188417,60361.042101,35.0,r,186.400863,0.0,1,1.093293,0.583235,...,22.244505,243.674224,116.356640,27.200841,63.657950,124.564260,78.589008,,BLOCK-306,2024-02-21 01:00:37.503022080+00:00


In [36]:
visits.groupby('science_program').count()

,fieldRA,fieldDec,observationStartMJD,visitExposureTime,filter,rotSkyPos,rotSkyPos_desired,numExposures,airmass,seeingFwhm500,...,rotTelPos_backup,moonAz,sunAz,moonRA,moonDec,moonDistance,solarElong,moonPhase,observation_reason,start_date
science_program,,,,,,,,,,,,,,,,,,,,,
BLOCK-305,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
BLOCK-306,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
BLOCK-312,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
BLOCK-T359_WD_0621-376,28,28,28,28,28,28,28,28,28,28,...,28,28,28,28,28,28,28,28,28,28
BLOCK-T359_WD_0859-039,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


In [37]:
with open(custom_tabs_fname, "r") as custom_tabs_file:
    custom_json = custom_tabs_file.read()

print(custom_json)

[
    {
        "name": "Slew Time",
        "settings": {
            "kind": "scatter",
            "x": "slewDistance",
            "y": "slewTime",
            "ylabel": "slew time (seconds)",
            "xlabel": "slew distance (degrees)",
            "color": "visitTime",
            "size": "visitExposureTime",
            "clabel": "visit time (seconds)",
            "cmap": "isolum"
        }
    },
    {
        "name": "Airmass histogram",
        "settings": {
            "kind": "hist",
            "y": "airmass",
            "by": "note",
            "bins": 8,
            "height": 512,
            "ylabel": "number of visits"
        }
    },
    {
        "name": "Altitude Plot by Target",
        "settings": {
            "kind": "scatter",
            "x": "start_date",
            "y": "altitude",
            "ymin": 20,
            "ymax": 90,
            "by": "science_program"
        }
    }
]


## Make the dashboard

Including two instances of the scheduler takes too much memory, crashes the kernel. Bummer.

prenight = schedview.app.prenight.Prenight()
pn_app = prenight.make_app(
    night_date,
    opsim_db=opsim_output_fname,
    scheduler=scheduler_fname,
    custom_hvplot_tab_settings_file=custom_tabs_fname,
    rewards=rewards_fname
)

show_inline = True

if show_inline:
    out = pn_app.show()
else:
    out = "Show with panel button at top of jupyter tab"

To open the schedview dashboard, run the command scheduler_dashboard from the terminal and select the schedule pickel file below. 

In [ ]:
scheduler.survey_lists[2]

In [ ]:
survey=scheduler.survey_lists[2][0]

In [ ]:
survey.basis_functions

In [ ]:
conditions=observatory.conditions

In [ ]:
conditions.mjd

In [ ]:
survey

In [ ]:
survey.make_reward_df(conditions)

In [ ]:
scheduler.request_observation(conditions)

In [ ]:
type(observatory)

In [ ]:
observatory.downtimes=np.array([])

In [ ]:
observatory.downtimes

In [ ]:
observatory.downtimes.clear()

In [ ]:
type(observatory.downtimes)

In [ ]:
observatory.check_up(60317.20)